## Этап 1: Загрузка данных

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('/datasets/insurance.csv')
data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [3]:
data.info()
data[data.duplicated() == True]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
281,1,39.0,48100.0,1,0
488,1,24.0,32900.0,1,0
513,0,31.0,37400.0,2,0
718,1,22.0,32600.0,1,0
785,0,20.0,35800.0,0,0
...,...,...,...,...,...
4793,1,24.0,37800.0,0,0
4902,1,35.0,38700.0,1,0
4935,1,19.0,32700.0,0,0
4945,1,21.0,45800.0,0,0


In [4]:
data[(data['Возраст'] == 39) & (data['Зарплата'] == 48100.0) & (data['Члены семьи'] == 1)]

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
69,1,39.0,48100.0,1,0
281,1,39.0,48100.0,1,0


In [5]:
data = data.drop_duplicates().reset_index(drop = True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4847 entries, 0 to 4846
Data columns (total 5 columns):
Пол                  4847 non-null int64
Возраст              4847 non-null float64
Зарплата             4847 non-null float64
Члены семьи          4847 non-null int64
Страховые выплаты    4847 non-null int64
dtypes: float64(2), int64(3)
memory usage: 189.5 KB


### Выводы  
Данные просмотрены.  
Изучен тип данных.    
Нашёл дубликаты, проверил ручками некоторые из них и удалил.  



## Этап 2: Алгоритм преобразования

**Алгоритм**

1) Создаём квадратную матрицу A из рандомных чисел с помощью np.random.randint(): её стороны равны количеству признаков датафрейма, подлежащего кодировке.  
2) Проверяем созданную матрицу на обратимость с помощью np.linalg.inv(). Если она обратима, получаем матрицу A^-1. Если нет, повторяем генерацию матрицы.  
3) Умножаем матрицу признаков на полученную квадратную матрицу и получаем закодированные данные.  
4) Для раскодирования используем умножение матрицы закодированных параметров на матрицу A^-1  
5) Проверяем качество раскодирования.  
6) Устраняем выявленные нарушения работы алгоритма.  

**Обоснование**

Умножение матрицы на обратимую квадратную матрицу изменит данные датафрейма, что защитит данные клиента.  
Также в наших руках будет ключ к дешифровке этой информации - обратная матрица.  
Данный подход не должен ухудшить качество модели линейной регрессии.  

## Этап 3: Проверка алгоритма

In [6]:
features = data.drop('Страховые выплаты', axis = 1)
target = data['Страховые выплаты']

In [7]:
from sklearn.model_selection import train_test_split

features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=12345)

In [45]:
state = np.random.RandomState(12345)
rand_matrix = state.randint(0, 100, size = (4, 4))
rand_matrix

array([[98, 29,  1, 36],
       [41, 34, 29,  1],
       [59, 14, 91, 80],
       [73, 11, 77, 10]])

In [9]:
inv_matrix = np.linalg.inv(rand_matrix)
res_matrix = pd.DataFrame(rand_matrix @ inv_matrix, columns = [0,1,2,3])
res_matrix

,0,1,2,3
0,1.000000e+00,1.772671e-17,1.084202e-17,-4.336809e-18
1,5.139118e-17,1.000000e+00,3.285133e-17,-6.310057e-17
2,-4.163336e-17,-4.076600e-17,1.000000e+00,-4.163336e-17
3,-3.942159e-16,-6.638028e-17,-3.252607e-18,1.000000e+00


In [10]:
mod_features_train = features_train @ rand_matrix
mod_features_valid = features_valid @ rand_matrix

In [11]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

In [12]:
model = LinearRegression()
model.fit(features_train, target_train)
model_predicted = model.predict(features_valid)

In [13]:
encoded_model = LinearRegression()
encoded_model.fit(mod_features_train, target_train)
encoded_models_predicted = encoded_model.predict(mod_features_valid)

In [14]:
r2score = r2_score(target_valid, model_predicted)
encoded_models_r2_score = r2_score(target_valid, encoded_models_predicted)
mse_normal = mean_squared_error(target_valid, model_predicted)
mse_encoded = mean_squared_error(target_valid, encoded_models_predicted)

print('Normal data R2 score:', r2score, 'MSE:', mse_normal)
print('Encoded data R2 score:', encoded_models_r2_score, 'MSE:', mse_encoded)

Normal data R2 score: 0.42307727492147296 MSE: 0.11955009374099915
Encoded data R2 score: 0.4230772749210392 MSE: 0.11955009374108903


Обучил модель линейной регрессии на исходных датайфреймах и закодированных.    
Получил практически идентичную метрику R2 для обоих случаев, как и MSE.    
Сама же метрика R2 выше нуля и ниже единицы - модель работает.    

In [15]:
features_train

,Пол,Возраст,Зарплата,Члены семьи
4460,0,28.0,54500.0,2
3773,1,38.0,50700.0,1
4561,0,24.0,28000.0,1
1385,0,47.0,38800.0,0
724,0,40.0,32500.0,1
...,...,...,...,...
3497,1,23.0,28200.0,1
3492,0,30.0,33700.0,1
2177,1,20.0,28900.0,1
3557,1,33.0,38800.0,1


In [16]:
returned_features_train = mod_features_train @ inv_matrix
returned_features_valid = mod_features_valid @ inv_matrix
returned_features_train

,0,1,2,3
4460,1.767612e-11,28.0,54500.0,2.000000e+00
3773,1.000000e+00,38.0,50700.0,1.000000e+00
4561,7.432006e-12,24.0,28000.0,1.000000e+00
1385,-2.017524e-12,47.0,38800.0,2.559124e-12
724,-8.875717e-13,40.0,32500.0,1.000000e+00
...,...,...,...,...
3497,1.000000e+00,23.0,28200.0,1.000000e+00
3492,1.532006e-11,30.0,33700.0,1.000000e+00
2177,1.000000e+00,20.0,28900.0,1.000000e+00
3557,1.000000e+00,33.0,38800.0,1.000000e+00


In [17]:
returned_features_train.columns = ['Пол', 'Возраст', 'Зарплата', 'Члены семьи'] 
returned_features_train = round(returned_features_train[['Пол', 'Возраст', 'Зарплата', 'Члены семьи']], 2)
features_train = round(features_train[['Пол', 'Возраст', 'Зарплата', 'Члены семьи']], 2)


for i in ['Пол', 'Возраст', 'Зарплата', 'Члены семьи']:
    print('Совпдадает ли признак', i, returned_features_train[i].isin(features_train[i]).unique())

Совпдадает ли признак Пол [ True]
Совпдадает ли признак Возраст [ True]
Совпдадает ли признак Зарплата [ True]
Совпдадает ли признак Члены семьи [ True]


## Выводы:  
Предложенный алгоритм работает.  
Удалось исходные данные закодировать.  
Обученные модели по исходным данным и закодироованным показали идентичные метрики по R2 и MSE.  
Удалось добиться декодирования без потери качества данных.  
